In [1]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os
import time



In [2]:
data=pd.read_pickle('scrape_data.pkl')

In [3]:
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Journey
0,2007,1,1,1,1232.0,WN,SMF,ONT,389,SMFONT
1,2007,1,1,1,1918.0,WN,SMF,PDX,479,SMFPDX
6,2007,1,1,1,1936.0,WN,SMF,PHX,647,SMFPHX
12,2007,1,1,1,2128.0,WN,SMF,SAN,480,SMFSAN
25,2007,1,1,1,2119.0,WN,SMF,SEA,605,SMFSEA


In [7]:
data1=data[0:100]

In [11]:


def skyscannerscraper(dep,arr,date, direct='true'):
    # Function scraping skyscanner.dk
    # dep: airport code for departure airport
    # arr: airport code for arrival airport
    # date: date of flight, format YYMMDD
    # direct: indicates whether direct flight is preferred
    driver = webdriver.Firefox(executable_path='/Users/Christian/Dropbox/Universitet/TSODAS/test/geckodriver')
    driver.implicitly_wait(8)
    wait = WebDriverWait(driver, 8)
    #urlbase = 'https://www.skyscanner.dk/transport/fly/'
    #urlpas = '/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    #urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    urlbase = 'https://www.skyscanner.com/transport/flights/'
    urlpas='/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    urldend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    data = []

    for depa,arri in zip(dep,arr):
        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend


        driver.get(url)
            
        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))
            
            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


            data.append({
                "price": price[1],
                "duration": duration[1],
                "origin": str(depa),
                "destination": str(arri),
                "day":0
                    })
        except (TimeoutException,StaleElementReferenceException):
            try:
                WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]
                
                data.append({
                "price": price[1],
                "duration": duration[1],
                "origin": str(depa),
                "destination": str(arri),
                "day":0
                    })    
            except (TimeoutException,StaleElementReferenceException):
                try:
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                    duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                    data.append({
                    "price": price[0],
                    "duration": duration[0],
                    "origin": str(depa),
                    "destination": str(arri),
                    "day":0
                        })  
                except (TimeoutException,StaleElementReferenceException) :
                    try:
                        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+1) + urlpas + direct + urlend


                        driver.get(url)
                            

                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                        duration = [dur.text for dur in driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div/div[4]/section[3]/div/div[1]/div[3]/div/table/tbody/tr/td[2]/button/span[3]")]


                        data.append({
                                "price": price[1],
                                "duration": duration[1],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                    })
                    except (TimeoutException,StaleElementReferenceException):
                        try:
                            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                            data.append({
                            "price": price[1],
                            "duration": duration[1],
                            "origin": str(depa),
                            "destination": str(arri),
                            "day":1
                                    })    
                        except (TimeoutException,StaleElementReferenceException):  
                            try:
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                data.append({
                                "price": price[0],
                                "duration": duration[0],
                                "origin": str(depa),
                                "destination": str(arri),
                                "day":1
                                     }) 
                            except (TimeoutException,StaleElementReferenceException):
                                try:
                                    url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date+2) + urlpas + direct + urlend


                                    driver.get(url)
                                        

                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                    duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]


                                    data.append({
                                            "price": price[1],
                                            "duration": duration[1],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                })
                                except (TimeoutException,StaleElementReferenceException):
                                    try:
                                        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-price")))
                                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fqs-duration")))

                                        price = [price.text for price in driver.find_elements(By.CLASS_NAME, "fqs-price")]
                                        duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "fqs-duration")]

                                        data.append({
                                        "price": price[1],
                                        "duration": duration[1],
                                        "origin": str(depa),
                                        "destination": str(arri),
                                           "day":2
                                                })    
                                    except (TimeoutException,StaleElementReferenceException):  
                                        try:
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "price")))
                                            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "LegInfo__duration-2VgVw")))

                                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "price")]
                                            duration = [dur.text for dur in driver.find_elements(By.CLASS_NAME, "LegInfo__duration-2VgVw")]


                                            data.append({
                                            "price": price[0],
                                            "duration": duration[0],
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":2
                                                 }) 
                                        except (TimeoutException,StaleElementReferenceException):  
                                            print("No working for"+str(url))
                                            data.append({
                                            "price": "-",
                                            "duration": "-",
                                            "origin": str(depa),
                                            "destination": str(arri),
                                            "day":"-"
                                                 }) 
        print(str(depa)+"-"+str(arri))
    driver.close()
    price_data = pd.DataFrame(data)                                            
    return price_data



In [9]:
prices=skyscannerscraper(data1['Origin'],data1['Dest'],190527, direct='true')

No working forhttps://www.skyscanner.dk/transport/fly/SMF/ONT/190529/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects=true&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results
SMF-ONT
SMF-PDX
SMF-PHX
SMF-SAN
SMF-SEA


KeyboardInterrupt: 

In [10]:
prices.head()

NameError: name 'prices' is not defined